# Demonstrate parallel execution with multiprocessing.Pool in Python

## Problem statement

Create a set of hashes from a large dictionary of words. A relatively slow hash algorithm is used for demonstration purposes. The example is taken from https://superfastpython.com/multiprocessing-pool-python/.

## Preparation

Download the data from https://raw.githubusercontent.com.
In a terminal window type 
```wget https://raw.githubusercontent.com/SuperFastPython/DataSets/main/bin/1m_words.txt.zip```
Then unzip the file with `unzip 1m_words.txt.zip`
You can remove the `__MACOS/` folder afterwards.


In [ ]:
# Define function that shall be executed in parallel
# Note: there is a certain overhead for creating the pool and executing threads. 
# Therefore, the function should be "reasonably complex" to achieve any speedup through parallelisation
from hashlib import sha512
# hash one word using the SHA algorithm
def hash_word(word):
    # create the hash object
    hash_object = sha512()
    # convert the string to bytes
    byte_data = word.encode('utf-8')
    # hash the word
    hash_object.update(byte_data)
    # get the hex hash of the word
    return hash_object.hexdigest()

In [ ]:
from multiprocessing import Pool
import os

In [ ]:
# print number of available CPUs
print(os.cpu_count())
# set number of processes for multiprocessing pool
processes = 8

In [ ]:
with open('1m_words.txt', 'r') as f:
    words = f.readlines()
print(f'Read {len(words)} words from file.')

In [ ]:
%%timeit -n 1 -r 5
# let's do a serial hashing first
known_words = {hash_word(w) for w in words}

In [ ]:
%%timeit -n 1 -r 5
# now we do the same with parallel processes
with Pool(processes) as p:
    known_words = set(p.map(hash_word, words))

In [ ]:
# in my trial runs, I find a speedup of ~20% using 4 or 8 processes (1.03 versus 1.19 seconds). 
# The program gets slower when I increase the number of processes to 32 or more.
# Note: a bug in IPython 7.4 prevents the result value being set correctly with %%timeit!
# Therefore, we need to execute the hashing once more to see the result
with Pool(processes) as p:
    known_words = set(p.map(hash_word, words))
print(list(known_words)[9000:9010])

## Asynchronous execution

Here, we will use asynchronous mapping so that the main process can continue during the function calls.
Note that results from the parallel tasks may not be avialble before all tasks have been executed, so the main program should not rely on them.

Pool.close and Pool.join can be used to signal the end of the parallel processing and wait for the results.

Again, the following example is from https://superfastpython.com/multiprocessing-pool-python/ with some modifications to illustrate continuation of the main program and an error callback function.

In [ ]:
# example of the map_async and forget usage pattern
from time import sleep
from random import random
 
# task to execute in a new process
def task(value):
    if value == 42 or value == 13:
        raise ValueError(f'{value} is an impossible value!')
    # generate a random value
    random_value = random()
    # block for moment
    sleep(random_value)
    # prepare result
    result = (value, random_value)
    # report results
    print(f'>task({result[0]}):{result[1]:.3f}', end='\n', flush=True)

# error callback function
def galactic_error(value):
    print(f'***{value}***')
    
# main task; run this while the parallel tasks are executing. Nothing fancy...
def main_task():
    sleep_time = 0.04
    for i in range(100):
        print(f'$main({i}):{sleep_time:.3f}', end='\n')
        sleep(sleep_time)
        
# protect the entry point
if __name__ == '__main__':
    # create the process pool
    with Pool(processes=8) as pool:
        # issue tasks to the process pool
        _ = pool.map_async(task, range(60), chunksize=16, error_callback=galactic_error)
        # continue execution of main program
        main_task()
        # close the pool
        pool.close()
        # wait for all tasks to complete
        pool.join()

## Summary

from https://superfastpython.com/multiprocessing-pool-python/

Use multiprocessing.Pool when

* Your tasks can be defined by a pure function that has no state or side effects.

* Your task can fit within a single Python function, likely making it simple and easy to understand.

* You need to perform the same task many times, e.g. homogeneous tasks.

* You need to apply the same function to each object in a collection in a for-loop.

Process pools work best when applying the same pure function on a set of different data (e.g. homogeneous tasks, heterogeneous data). This makes code easier to read and debug.

Donot Use multiprwcessing.Pool When

* You have a single task; consider using the Process class with the "target" argument.

* You have long running tasks, such as monitoring or scheduling; consider extending the Process class.

* Your task functions require state; consider extending the Process class.

* Your tasks require coordination; consider using a Process and patterns like a Barrier or Semaphore.

* Your tasks require synchronization; consider using a Process and Locks.

* You require a process trigger on an event; consider using the Process class.

The sweet spot for process pools is in dispatching many similar tasks, the results of which may be used later in the program. Tasks that don't fit neatly into this summary are probably not a good fit for process pools.



In [52]:
x=5